# 基盤人工知能演習 第6回

※本演習資料の二次配布・再配布はお断り致します。


　今回の演習は以下の通りである。

　**AI6.1 | MNISTを用いた多クラス分類**

　**AI6.2 | 勾配消失問題とその対策**

　**AI6.3 | Dropoutによる正則化**

## AI6.0 | 事前準備

### AI6.0.1 | 手書き文字 MNIST の準備

　今回は手書き文字 MNIST に対する学習を題材に、多層パーセプトロン (multi-layer perceptron) による多クラス分類 (multi-class classification) を学んでいく。MNIST は有名なデータセットで、いろんな人が機械学習（特にニューラルネットワーク）のテスト実行の題材として利用している。

　PyTorchには、 `torchvision` という画像処理向けのライブラリが別途用意されており、これを使うことでMNISTデータを自動的に処理することができる。

In [0]:
import numpy as np
import torch
import torchvision

trainset = torchvision.datasets.MNIST(root=".", train=True, download=True, transform=torchvision.transforms.ToTensor())
testset  = torchvision.datasets.MNIST(root=".", train=False, download=True, transform=torchvision.transforms.ToTensor())

In [0]:
# torch.tensor -> numpy.array
X_train = trainset.data.numpy()
y_train = trainset.targets.numpy()
X_test = testset.data.numpy()
y_test = testset.targets.numpy()

# scaled from [0,255] to [0,1] for X
X_train = X_train / 255
X_test  = X_test / 255

　torchvisionの `torchvision.datasets.MNIST` 関数は `torch.tensor` に自動的に変換してくれるのだが、中身のデータを目視で確認するために、一旦 `numpy.array` に変換した。

　MNISTのデータは、以下のような件数のデータが含まれている。

+ `X_train`: $60000 \text{ (images)} \times 28 \text{ (rows, y)} \times 28 \text{ (columns, x)}$
+ `y_train`: $60000 \text{ (labels)}$
+ `X_test`: $10000 \text{ (images)} \times 28 \text{ (rows, y)} \times 28 \text{ (columns, x)}$
+ `y_test`: $10000 \text{ (labels)}$


　実際にPythonのコードを書いて確認してみよう。

In [0]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

　次に、MNISTの画像について、いくつか描画してみよう。
`i` の値を書き換えることで、いろいろな手書き文字を見ることができる。
また、各座標には0.0から1.0までの値が記述されているが、これが入力特徴量になる。

　目視してみると、基盤DS演習で用いている手書き文字よりも解像度が高いことがわかる。あちらで利用した手書き文字データは8×8だったが、MNISTは28×28なので、一辺が3倍以上増えている。


In [0]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Index number of an instance (change this to view another instance).
i = 1024

image = X_train[i]
label = y_train[i]

print(label)
plt.figure(figsize=(16, 16))
sns.heatmap(image, annot=True, fmt='.1f', square=True, cmap="YlGnBu")
plt.show()

### AI6.0.2 | livelossplotのインストール

　今回のMNISTは6万件ものデータが含まれているため、学習にそれなりの時間がかかる。リアルタイムで学習の状況を見る1つの方法として、livelossplotというものを利用してみる。

　livelossplotは初期状態ではインストールされていないので、以下のコマンドを利用してインストールを行う。

In [0]:
# 以下のコマンドはPythonのコマンドではなく、Google Colab固有の方法であることに注意
# pipというPythonのパッケージ管理ツールを使ってlivelossplotをインストールしている
!pip install livelossplot

　色々な文字列が出て来るが、最終的に **"Successfully installed ..."** のような記述がされていればインストールは完了である。（もし **"Requirement already satisfied"** で終わっていたら、既にインストールは完了していると考えてよい）

## AI6.1 | MNISTを用いた多クラス分類

　前回の演習（基盤人工知能演習 第5回）では、クラス0（×）とクラス1（○）の2クラス分類を行った。一方、**今回使うMNISTは0から9までの10種類の数字があり、それぞれを予測する**。このように、3つ以上のクラスを同時に予測することを**多クラス分類 (multi-class classification)** と呼ぶ。

　2クラス分類の時には、入力 $x$ に対応する出力 $y$ はスカラ量であり、クラス1（○）である確率を出力していた。
しかし、 $K$ クラス分類の場合は、出力値は $K$ 次元のベクトル $y=[y_1, y_2, ..., y_K]$ にして、$y_i$は入力 $x$ がクラス $i$ である確率と考える。

### AI6.1.1 | PyTorchのためのデータ準備

　それでは、実際に学習を行うために、いろいろな作業を行おう。

　最初に、`X_train`, `y_train`を`X_train`, `X_valid`, `y_train`, `y_valid`に分割する（training-validation分割）。基盤データサイエンス 第3回 では交差検証法 (cross validation) を学んだが、ニューラルネットワークでは、計算時間の都合から交差検証法をしないことが多い。

In [0]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
# trainを50000件のtrainingデータと10000件のvalidationデータに分割する
# 分割結果が毎回同じになるように、random_state=0を付与している
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, 
                                                      train_size=50000, random_state=0)

# testはそのまま利用する

　続いて、前回と同様にXをtorch用のデータに変換する。`X` は先ほど見たように2次元状にデータが並んでいるが、通常のニューラルネットワークは2次元形式で与えられても処理できないため、 $28 \times 28$ の要素がある1次元配列に変換し、それからPyTorch形式に変換する。

In [0]:
X_train_flatten = X_train.reshape(50000, 28*28) # 50000件の2次元画像（28×28）を50000件の1次元のデータ（784）に変換
X_valid_flatten = X_valid.reshape(10000, 28*28)
X_test_flatten  = X_test.reshape(10000, 28*28) 

In [0]:
X_train_torch = torch.tensor(X_train_flatten, dtype=torch.float) # dtype=torch.floatを忘れずに
X_valid_torch = torch.tensor(X_valid_flatten, dtype=torch.float) 
X_test_torch  = torch.tensor(X_test_flatten,  dtype=torch.float) 

　次に、$y$ をPyTorch用のデータ形式にする。

In [0]:
y_train_torch = torch.tensor(y_train, dtype=torch.long) # dtype=torch.longを忘れずに
y_valid_torch = torch.tensor(y_valid, dtype=torch.long) 
y_test_torch  = torch.tensor(y_test,  dtype=torch.long)

　これで基本準備ができた。そうしたら、あとは前回と同様にdatasetとdataloaderを定義して、モデルを作って学習を行ってみるだけだ。ただし、datasetとdataloaderはtrainだけではなくvalid, testも別に用意しよう。

In [0]:
# XとYの組み合わせを保持してくれる便利なモノ
train_dataset = torch.utils.data.TensorDataset(X_train_torch, y_train_torch)
valid_dataset = torch.utils.data.TensorDataset(X_valid_torch, y_valid_torch)
test_dataset  = torch.utils.data.TensorDataset(X_test_torch,  y_test_torch)

In [0]:
batch_size = 256 # 今回は256件のデータごとに学習を行う

# XとYの組み合わせを(batch_size)個ずつ出力する便利なモノ
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size)
test_loader  = torch.utils.data.DataLoader(test_dataset,  batch_size=batch_size)

### AI6.1.2 | MNISTの学習の実行

　それでは、ここまでに作成したDataLoaderを利用して、学習を行ってみる。ここでは、モデル `model` 、誤差関数 `loss_fn`、および最適化手法 `opt` を引数とすることで、汎用的に学習や予測を行えるコードを作成してみる。

　作ろうとするコードは多少複雑になるので、これからやることを、まず箇条書きする。

1. 訓練データ `train_loader` を用いてモデルのパラメータを更新する。その時に、訓練誤差（訓練データに対する誤差関数の値）と訓練データに対する予測精度を保存しておく。
2. 検証データ `valid_loader` を用いて、現在のモデルがどの程度の汎化性能 / 汎化誤差を持つか推定する。
3. `liveloss` を利用して、1.および2.で計算した誤差および精度を描画する。
4. 1.から3.をepoch数だけ繰り返す。

箇条書きをしたら、それに対応するコードを関数として記述してみよう。


In [0]:
import torch
from livelossplot import PlotLosses

device_GPU = torch.device("cuda:0") 

In [0]:
# 箇条書き 1. に対応
def update_model(model, loss_fn, opt, train_loader, device):
  train_loss = 0
  train_correct = 0
  train_count = len(train_loader.dataset)
  
  for X, y in train_loader:
    X = X.to(device) # GPUへデータを転送
    y = y.to(device) # GPUへデータを転送
    y_pred = model(X) # Xからyを予測（softmaxを行う前の値が出力される）
    
    _, predicted = torch.max(y_pred.data, 1) # 10クラスのうち、予測確率最大のクラス番号を取得
    train_correct += (predicted == y).sum().item() # 予測に成功した件数をカウント（accuracy計算用）
    
    loss = loss_fn(y_pred, y)        # ミニバッチ内の訓練誤差の 平均 を計算
    train_loss += loss.item()*len(y) # エポック全体の訓練誤差の 合計 を計算しておく
    
    # 重みの更新
    opt.zero_grad()
    loss.backward()
    opt.step()
    
  # エポック内の訓練誤差の平均値と予測精度を計算
  mean_train_loss = train_loss / train_count
  train_accuracy = train_correct / train_count
  
  return mean_train_loss, train_accuracy    

In [0]:
# 箇条書き 2. に対応
def evaluate_model(model, loss_fn, dataloader, device):
  model.eval() # 学習を行わない時は evaluate 状態にする （補足資料※1）

  valid_loss = 0
  valid_correct = 0
  valid_count = len(dataloader.dataset)

  for X, y in dataloader:
    X = X.to(device) # GPUへデータを転送
    y = y.to(device) # GPUへデータを転送
    y_pred = model(X) # Xからyを予測（softmaxを行う前の値が出力される）
    
    _, predicted = torch.max(y_pred.data, 1) # 10クラスのうち、予測確率最大のクラス番号を取得
    valid_correct += (predicted == y).sum().item() # 予測に成功した件数をカウント（accuracy計算用）
    
    loss = loss_fn(y_pred, y)        # ミニバッチ内の訓練誤差の 平均 を計算
    valid_loss += loss.item()*len(y) # エポック全体の訓練誤差の 合計 を計算しておく
    
  mean_valid_loss = valid_loss / valid_count
  valid_accuracy = valid_correct / valid_count

  model.train() # evaluate状態からtrain状態に戻しておく
  return mean_valid_loss, valid_accuracy

In [0]:
# 箇条書き 3. および 4. に対応
def train(model, loss_fn, opt, train_loader, valid_loader, device, epoch=50):
  liveloss = PlotLosses() # 描画の初期化
  for i in range(epoch):
    train_loss, train_accuracy = update_model(model, loss_fn, opt, train_loader, device)
    valid_loss, valid_accuracy = evaluate_model(model, loss_fn, valid_loader, device)
  
    # Visualize the loss and accuracy values.
    liveloss.update({
        'log loss': train_loss,
        'val_log loss': valid_loss,
        'accuracy': train_accuracy,
        'val_accuracy': valid_accuracy,
    })
    liveloss.draw()  
  print('Accuracy: {:.4f} (valid), {:.4f} (train)'.format(valid_accuracy, train_accuracy))
  return model # 学習したモデルを返す

最後に単層パーセプトロンのモデルを作成する。

In [0]:
torch.manual_seed(0) # 学習結果の再現性を担保

slp = torch.nn.Sequential(
    torch.nn.Linear(28*28, 10) # 10クラス分類なので出力は10次元
)
slp.to(device_GPU) # deviceへモデルを転送

# 誤差関数と最適化手法を準備
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(slp.parameters(), lr=0.1)

　ここで作成したモデルを**図 AI6.1**に示す。

![図 AI6.1](https://i.imgur.com/WpihxBG.png)

**図 AI6.1 | 単層パーセプトロン** 最後のsoftmaxは `torch.nn.CrossEntropyLoss()` の中で定義されている。

　softmaxはsigmoid関数の多クラス分類への拡張であり、以下の式で表すことができる。

$\begin{eqnarray}
\mathrm{softmax}{(z)}_k & = & \frac{e^{z_k}}{\sum^9_{i=0}e^{z_i}}
\end{eqnarray}$

このsoftmaxで変換された $\hat{y}_k$ は**確率の要件である $0 \le \hat{y}_k \le 1$ かつ $\sum_k \hat{y}_k = 1$
を満たす**ので、入力されたデータが0~9のどの数字っぽいか、の予測確率であると考えることができる。

　また、交差エントロピー誤差 `torch.nn.CrossEntropyLoss()` についても多クラス分類の拡張が行われている。

$\begin{eqnarray}
H(y, \hat{y}) & = & -\left(\sum^9_{k=0} y_k\log\hat{y}_k\right) \\
y_k & = & \left\{
\begin{array}{l}
      0\ \ (y\ne k) \\
      1\ \ (y=k)
    \end{array}
  \right.
\end{eqnarray}$

上記の $y_k$ のように、$y=2$ を $[0,0,1,0,0,...]^T$ とベクトルで表現したものを **one-hot vector** と呼ぶ（1か所だけ非ゼロの値になっていることを one-hot と表現している）。
$H(y, \hat{y})$は総和で記述されているが、one-hot vectorの特性から $H(y, \hat{y}) = -\log\hat{y}_k$ で計算できる。



　それでは、学習を行ってみよう。数分かかるが、epoch数が増えるにしたがって予測精度が変化していく様を見ると面白い。

In [0]:
# 学習の実行
trained_model = train(slp, loss_fn, optimizer, train_loader, valid_loader, device_GPU)

　最後に作成されたモデルを使って、テストデータを予測してみよう（本当は検証 (validation) データで精度最大であったようなepochの時のモデルを利用すべきなのだが、ここでは50epoch後のモデルを使ってテストデータの予測を行う）。

In [0]:
test_loss, test_accuracy = evaluate_model(slp, loss_fn, test_loader, device_GPU)
print(test_loss)
print(test_accuracy)

　どうやら、単層パーセプトロンを用いてだいたい92%程度の予測が行えるようだ。

-----
##### 課題 AI6.1

　これまでのコードで、784次元の画像データから10クラスの確率を出力する**単層の**パーセプトロンを構築した。

　今度は**多層の**パーセプトロンを構築し、予測を行ってみよう。（入力 28×28 → 512 → 256 → 10 出力）となるような多層パーセプトロン（**図 AI6.2**）を構築し、実際に学習を行うことで、50 epoch後の訓練データ、検証データに対する正解率 (accuracy) 、およびテストデータに対する正解率 (accuracy) を答えよ。

![図 AI6.2](https://i.imgur.com/llSr2UW.png)

**図 AI6.2 | 多層パーセプトロン** 最後のsoftmaxは `nn.CrossEntropyLoss()` の中で定義されている。

In [0]:
torch.manual_seed(0) # 学習結果の再現性を担保

mlp = torch.nn.Sequential(
  torch.nn.Linear(__number__, __number__), 
  torch.nn.Sigmoid(),
  torch.nn.Linear(__number__, __number__), 
  torch.nn.Sigmoid(),
  torch.nn.Linear(__number__, __number__), 
)
mlp.to(device_GPU) # deviceへモデルを転送

# 誤差関数と最適化手法を準備
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(mlp.parameters(), lr=0.1)

# 学習の実行
trained_model = train(mlp, loss_fn, optimizer, train_loader, valid_loader, device_GPU)

# 予測の実行
test_loss, test_accuracy = evaluate_model(mlp, loss_fn, test_loader, device_GPU)
print(test_loss)
print(test_accuracy)

------
##### 課題 AI6.2（発展、提出する必要はありません）

　`update_model()` 関数と `evaluate_model()` 関数は、よくよく見てみると非常に似ている。これらを関数をまとめて、引数に応じてモデルの更新およびモデルの評価を行う `calculate_model()` 関数を作成せよ。

　関数を作成したら、この関数を用いた学習を行い、結果がほとんど変わらないことを確認せよ。

---------

## AI6.2 | 勾配消失問題とその対策

　ニューラルネットワークは、多層にすることで複雑な関数を表現できることを学んできた（直前の演習では、単層に比べて多層にすることで精度が向上するはずである）。

　一方、層をとりあえず深くしておけば、精度が向上するとは限らない。層が深くなりすぎた場合に、誤差逆伝播が殆ど機能しなくなる、「**勾配消失問題**」という問題があるためである。

　実際に、8層のモデルを利用して試してみよう。これはかなり層が深いモデルであると言える。

In [0]:
###### 最終的な結果を毎回同じ値にするおまじない ######
torch.manual_seed(0)

model = torch.nn.Sequential(
  torch.nn.Linear(28*28, 512), 
  torch.nn.Sigmoid(),
  torch.nn.Linear(512, 512), 
  torch.nn.Sigmoid(),
  torch.nn.Linear(512, 512), 
  torch.nn.Sigmoid(),
  torch.nn.Linear(512, 512), 
  torch.nn.Sigmoid(),
  torch.nn.Linear(512, 512), 
  torch.nn.Sigmoid(),
  torch.nn.Linear(512, 512), 
  torch.nn.Sigmoid(),
  torch.nn.Linear(512, 256), 
  torch.nn.Sigmoid(),
  torch.nn.Linear(256, 10), 
)
model.to(device_GPU) # deviceへモデルを転送

# 誤差関数と最適化手法を準備
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

# 学習の実行
trained_model = train(model, loss_fn, optimizer, train_loader, valid_loader, device_GPU)

# 予測の実行
test_loss, test_accuracy = evaluate_model(model, loss_fn, test_loader, device_GPU)
print(test_loss)
print(test_accuracy)

　このように、学習がほとんど進まないという状況が発生する。これは、**シグモイド関数の微分値の最大値が1/4**であり、さらに**微分値がほぼ0である領域が非常に広い**ことに起因している（cf. **図 AI6.3**のように、**誤差逆伝播の計算における計算グラフの処理は掛け算の繰り返し**であることを思い出せ）。

　これを回避する方法として、ReLU (Rectified Linear Unit) という活性化関数がよく用いられている。これは $\mathrm{ReLU}(x) = \max(0, x)$ で表される関数で、**$x$ が正であれば微分値はどこまでも1**である（**図 AI6.4**）。


![図 AI6.3](https://i.imgur.com/VsfdLcp.png)

**図 AI6.3 | 誤差逆伝播** 図 AI5.2と同一図を再掲している。$\Sigma, \sigma, H$ はそれぞれ総和、シグモイド関数、交差エントロピーを意味する。


![図 AI6.4](https://i.imgur.com/vZ3edOO.png)

**図 AI6.4 | Sigmoid関数とReLU関数** 関数の傾きの差を見るために、同一縮尺で描画した。


　同じ8層のモデルについて、 `Sigmoid()` の代わりに `ReLU()` を使って学習させてみる。なお、ReLUの場合はSigmoidを使う場合に比べて少し学習率を下げておくと良いだろう。

In [0]:
###### 最終的な結果を毎回同じ値にするおまじない ######
torch.manual_seed(0)

model = torch.nn.Sequential(
  torch.nn.Linear(28*28, 512), 
  torch.nn.ReLU(),
  torch.nn.Linear(512, 512), 
  torch.nn.ReLU(),
  torch.nn.Linear(512, 512), 
  torch.nn.ReLU(),
  torch.nn.Linear(512, 512), 
  torch.nn.ReLU(),
  torch.nn.Linear(512, 512), 
  torch.nn.ReLU(),
  torch.nn.Linear(512, 512), 
  torch.nn.ReLU(),
  torch.nn.Linear(512, 256), 
  torch.nn.ReLU(),
  torch.nn.Linear(256, 10), 
)
model.to(device_GPU) # deviceへモデルを転送

# 誤差関数と最適化手法を準備
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.02) # 学習率を1/5に下げている 

# 学習の実行
trained_model = train(model, loss_fn, optimizer, train_loader, valid_loader, device_GPU)

# 予測の実行
test_loss, test_accuracy = evaluate_model(model, loss_fn, test_loader, device_GPU)
print(test_loss)
print(test_accuracy)

　最初こそ予測精度は向上していなかったが、20epochあたりから急激に正解率が向上、適切なモデルが構築できていることがわかる。ただ、時々大きく汎化性能が悪化することがあるため、どのepochが最も汎化性能が高いかをしっかり把握しておく必要がある（**補足資料 ※2**）。 `train()` 関数の中で、最も精度が良かったモデルを記録しておくなどのコードの修正が必要だろう。

-----
##### 課題 AI6.3

　`torch.nn.ReLU()` 関数の派生として、`torch.nn.PReLU()` という活性化関数が存在する。どのような活性化関数か調べて簡潔にまとめよ。

　また、前述のモデルについて、全ての活性化関数を`torch.nn.PReLU()`に取り換えて学習を行い、テストデータに対する正解率 (accuracy) を答えよ。

In [0]:
###### 最終的な結果を毎回同じ値にするおまじない ######
torch.manual_seed(0)

model = torch.nn.Sequential(
    # 課題 AI6.4: 実装せよ
)

model.to(device_GPU) # deviceへモデルを転送

# 誤差関数と最適化手法を準備
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.02)

# 学習の実行
trained_model = train(model, loss_fn, optimizer, train_loader, valid_loader, device_GPU)

# 予測の実行
test_loss, test_accuracy = evaluate_model(model, loss_fn, test_loader, device_GPU)
print(test_loss)
print(test_accuracy)

-----

## AI6.3 | Dropoutによる正則化

　活性化関数 `ReLU()` を用いることで、層を深くすることに成功した。一般に、層を深くした方が複雑な関数を表現しやすくなる。
一方で、複雑な関数を表現できるということは、過学習 (over-fitting) を起こしやすい、ということでもある。

　基盤人工知能 第3回では過学習への対応策としては**正則化**というものが存在し、Ridge回帰を行うことで過学習をある程度防ぐことができるということを学んだ。
今回はニューラルネットワークでよく用いられる過学習防止手法の1つであるDropoutを利用してみる。


　Dropoutの導入自体は非常に簡単であり、modelの**活性化関数の後に**導入すればよい。ただし、なんでもかんでも入れればよいというわけではなく、**適切に**入れる必要がある（がこれは非常に難しい）。
今回は、何も検討せずに、3層のニューラルネットワークの全ての活性化関数の後にDropoutを導入してみよう。


In [0]:
###### 最終的な結果を毎回同じ値にするおまじない ######
torch.manual_seed(0)

model = torch.nn.Sequential(
  torch.nn.Linear(28*28, 512), 
  torch.nn.ReLU(),
  torch.nn.Dropout(p=0.5),
  torch.nn.Linear(512, 256), 
  torch.nn.ReLU(),
  torch.nn.Dropout(p=0.5),
  torch.nn.Linear(256, 10), 
)
model.to(device_GPU) # deviceへモデルを転送

# 誤差関数と最適化手法を準備
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

# 学習の実行
trained_model = train(model, loss_fn, optimizer, train_loader, valid_loader, device_GPU)

# 予測の実行
test_loss, test_accuracy = evaluate_model(model, loss_fn, test_loader, device_GPU)
print(test_loss)
print(test_accuracy)

　**Dropoutを導入することで、訓練データに対する精度が抑制され、過学習が抑えられる**。一方、**Dropoutは学習速度を低下させる**ため、Dropoutを導入して学習を行う場合には、epoch数を増加させたり、学習率を上げることを検討する必要がある。

-------
##### 課題 AI6.4

　直前に作成したモデルからDropoutを1つ抜いたモデルを2種類構築し、Dropout層を2つ入れた場合のモデルと比較し、最良のモデルを3種類のモデルの中から答えよ。レポートには、「最良」であると考えた理由も含めて答えること。

-------

# レポート提出について



## レポートの提出方法
　レポートは**答案テンプレートを用い**、**1つのファイル (.doc, .docx, .pdf)** にまとめ、**学籍番号と氏名を確認の上**、**1/23 15:00 (次回 基盤人工知能演習) までに東工大ポータルのOCW-iから提出**すること。
ファイルのアップロード後、OCW-iで「提出済」というアイコンが表示されていることを必ず確認すること。それ以外の場合は未提出扱いとなるので十分注意すること。
また、締め切りを過ぎるとファイルの提出ができないため、時間に余裕を持って提出を行うこと。

## 答案テンプレート

```
学籍番号:
名前:

課題 AI6.1
訓練データに対する正解率：
検証データに対する正解率：
テストデータに対する正解率：

課題 AI6.3
PReLUとは：
テストデータに対する正解率：

課題 AI6.4
最良のモデル：
理由：
```



# 補足資料


## ※1 なぜ `evaluate()` 時には`model.eval()`が必要なのか
　Dropoutなど、学習と予測で異なる動作をする層が存在する（Dropoutの挙動については基盤人工知能 第6回 資料参照のこと）。そのため、重みの更新を行わないようにするだけでは不十分であり、`eval()`状態にする必要がある。

## ※2 `ReLU()` を用いる時の注意点

　実際の実行で示したように、`ReLU()`を用いることで学習がかなり効率化されている。一方、予測結果がしばしば極端になるようで、例えば99.99%以上の極めて高確率で正例、と予測したデータが実際には負例であった時などに誤差関数の値がとても大きな値になり、一時的に予測精度が低下したり、場合によっては学習が発散してしまう。

　このような現象があるため、学習が正しく進行しているかどうか適宜確認し、うまくいっていない場合には学習率を下げるなどの対応をしよう。